# Objectives

In this notebook I list various objective functions for optimization problems, and I group similar functions together. At the bottom there's also the random search class, so you can optimize any objective listed here by setting it to "obj" and modifying random search's ranges. By default I set the semi-empirical mass formula as "obj".

In [1]:
import random as r

## Random Search

In [2]:
class RandomSearch:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
        
    def generateRandomHypothesis(self):
        output = []
        for x in self.ranges:
            output.append(r.uniform(x[0], x[1]))
        return(output)
    
    def optimize(self,
                objectiveFunction,
                numIterations = 25000):
        bestX = None
        bestY = -float("inf")
        for i in range(numIterations):
            newX = self.generateRandomHypothesis()
            newY = objectiveFunction(newX)
            if newY > bestY:
                bestX = newX
                bestY = newY
        return(bestX)

## Physics Problems

### Drag Minimization 

The drag per second a vehicle travelling on a road experiences is approximately R(v) = Av^2 + Bv + C where A is a factor relating aerodynamics, B to rolling and drivetrain losses, and C is a constant load. If t is the time taken to arrive at a destination D, minimizing t x R will minimize the losses due to drag for the journey. Since v = D/t and D is constant this is equivalent to minimizing R/v, i.e. Av + B + C/v. 

Suppose we have a car with A = 0.6, B = 5, C = 80, we have a well-defined optimization problem. It's fun to play around with A/B/C and see how different physical parameters affect the optimum speed. most cars travel ~30mph most of the time, so we expect an answer near 13m/s

In [3]:
def dragMinimization(
    hypothesis,
    aerodynamicsConstant = 0.6,
    rollingConstant = 5,
    loadConstant = 80
):
    v = hypothesis[0]
    aTerm = aerodynamicsConstant * v 
    bTerm = rollingConstant
    cTerm = loadConstant / v
    output = - (aTerm + bTerm + cTerm)
    return(output)

In [4]:
rs = RandomSearch([[0, 50]])
print(rs.optimize(dragMinimization))

[11.547810193386631]


### Semi-Empirical Masss Formula (SEMF)

Below is the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

In [5]:
def SEMF(
    hypothesis, 
    volumeConstant = 15.8,
    surfaceConstant = 17.8,
    coulombConstant = 0.711,
    asymmetryConstant = 23.7,
    pairingConstant = 11.18
):
    
    #Physical info from hypothesis
    Z = int(hypothesis[0]) #Proton number
    N = int(hypothesis[1]) #Neutron number
    A = Z + N              #Nucleon number
    
    #Calculate each term
    volumeTerm = volumeConstant * A
    surfaceTerm = -surfaceConstant * A ** (2/3)
    coulombTerm = -coulombConstant * Z * (Z - 1) * A ** (-1/3)
    asymmetryTerm = -asymmetryConstant * ((N - Z) ** 2) / A
    
    #Pairing term
    if A % 2 == 1:
        pairingTerm = 0
    elif Z % 2 == 0 and N % 2 == 0:
        pairingTerm = pairingConstant * A ** (-1/2)
    elif Z % 2 == 1 and N % 2 == 1:
        pairingTerm = -pairingConstant * A ** (-1/2)
    else:
        pairingTerm = 0
    output = (volumeTerm + surfaceTerm + coulombTerm + asymmetryTerm + pairingTerm) / A
    return(output)

rs = RandomSearch([[1, 120], [1, 180]])
print(rs.optimize(SEMF))

[26.23921174602158, 32.26284512105376]


## Game Theory Problems

### El Farol Bar

With El Farol Bar, we assume there are several bars in town, say 5. Each bar has a base "niceness" value N, and we're antisocial nerds so we prefer bars with fewer people in. Thus the utility we would get for attending each bar is N^-kx where x is the number of other people in the bar and k some decay constant. 

Classic El Farol asks us to come up with a strategy for which bar to go to given other people are more likely to go to nicer bars, but I prefer to ask what the optimal distribution of people across bars is to maximise their culmulative utility. Thus we maximise the sum of xN^-kx for each N, x. (For convenience, k is constant across all bars).

We enforce that the number of people in each bar must be an integer, and the total number of people in all bars is free to vary (some people stay home and write code for GitHub instead).

In [53]:
barUtilities = [10, 20, 40, 80, 160]
decayConstant = 0.5

def singleBarUtility(x, barNumber):
    output = x * barUtilities[barNumber] ** (-decayConstant * (x - 1))
    return(output)

def elFarol(hypothesis):
    peopleAssignments = [int(x) for x in hypothesis]
    output = 0
    for i in range(len(barUtilities)):
        output += singleBarUtility(peopleAssignments[i], i)
    return(output)

In [54]:
ranges = []
for i in range(len(barUtilities)):
    ranges.append([0, 50])

rs = RandomSearch(ranges)
distribution = rs.optimize(elFarol)
print([int(x) for x in distribution])

[33, 45, 1, 1, 1]
